In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
pd.set_option('display.max.columns',None)
pd.options.display.max_colwidth = 100

In [2]:
# Uploading Edinburgh bikes data file.
edbikes_df = pd.read_csv('edinburgh_bikes_202205172048.csv',delimiter=';', decimal=',',index_col=0)
edbikes_df.head()

,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
index,,,,,,,,,,,,,
0,2018-09-15 08:52:05,2018-09-15 09:11:48,1182,247,Charlotte Square,North Corner of Charlotte Square,55.95233546161639,-3.207101172107286,259,St Andrew Square,North East corner,55.95472778937899,-3.192652969253117
1,2018-09-15 09:24:33,2018-09-15 09:41:09,995,259,St Andrew Square,North East corner,55.95474881217642,-3.192773668658787,262,Canonmills,near Tesco's,55.96280408759764,-3.196283585062929
2,2018-09-15 09:48:54,2018-09-15 10:46:40,3466,262,Canonmills,near Tesco's,55.96280408759764,-3.196283585062929,250,Victoria Quay,Entrance to Scottish Government Office,55.977637787726984,-3.174115590621568
3,2018-09-16 12:01:36,2018-09-16 12:25:26,1430,255,Kings Buildings 4,X-Y Cafe,55.92200087060626,-3.1769018141649212,254,Kings Building 3,Kings Building House,55.923478807846635,-3.175384584907078
4,2018-09-16 12:03:43,2018-09-16 12:11:16,452,255,Kings Buildings 4,X-Y Cafe,55.92200087060626,-3.1769018141649212,253,Kings Building 2,Sanderson Building,55.92320215516664,-3.17164624536349


In [5]:
# Data exploration: time frame. Table contains Edinburgh bikes borrowings from September 15th, 2018 until end June 2021.

edbikes_df[['started_at','ended_at']].sort_values('started_at')

,started_at,ended_at
index,,
0,2018-09-15 08:52:05,2018-09-15 09:11:48
1,2018-09-15 09:24:33,2018-09-15 09:41:09
2,2018-09-15 09:48:54,2018-09-15 10:46:40
3,2018-09-16 12:01:36,2018-09-16 12:25:26
4,2018-09-16 12:03:43,2018-09-16 12:11:16
...,...,...
12636,2021-06-30 23:30:31,2021-07-01 00:06:10
12637,2021-06-30 23:36:16,2021-07-01 00:05:40
12638,2021-06-30 23:49:03,2021-07-01 00:11:25


In [7]:
# Start stations. There are 198 stations at least once used for start.
start = (edbikes_df[['start_station_id','start_station_name','start_station_latitude','start_station_longitude']].drop_duplicates(subset='start_station_id')
.rename(columns={'start_station_id':'station_id','start_station_name':'station_name','start_station_latitude':'latitude','start_station_longitude':'longitude'})
)
print('Start stations:')
print('\n',start)

Start stations:

        station_id              station_name           latitude  \
index                                                            
0             247          Charlotte Square  55.95233546161639   
1             259          St Andrew Square  55.95474881217642   
2             262                Canonmills  55.96280408759764   
3             255         Kings Buildings 4  55.92200087060626   
5             253          Kings Building 2  55.92320215516664   
...           ...                       ...                ...   
4659         1860     Ingliston Park & Ride   55.9387919459937   
579          2259          Leith Walk North  55.96791807044289   
15127        2263          Musselburgh Lidl  55.94388031687606   
21035        2265  Musselburgh Brunton Hall  55.94400897108537   
5853         2268              Picady Place  55.95653524179326   

                 longitude  
index                       
0       -3.207101172107286  
1       -3.192773668658787  
2      

In [8]:
# End stations. There are 199 stations at least once used to end the borrowing.
end = (edbikes_df[['end_station_id','end_station_name','end_station_latitude','end_station_longitude']].drop_duplicates(subset='end_station_id')
       .rename(columns={'end_station_id':'station_id','end_station_name':'station_name','end_station_latitude':'latitude','end_station_longitude':'longitude'})
      )
print("End stations:")
print('\n',end)

End stations:

        station_id              station_name            latitude  \
index                                                             
0             259          St Andrew Square   55.95472778937899   
1             262                Canonmills   55.96280408759764   
2             250             Victoria Quay  55.977637787726984   
3             254          Kings Building 3  55.923478807846635   
4             253          Kings Building 2   55.92320215516664   
...           ...                       ...                 ...   
12197        1859    Edinburgh Park Central  55.931168715771065   
855          2259          Leith Walk North   55.96791807044289   
14980        2263          Musselburgh Lidl   55.94388031687606   
21035        2265  Musselburgh Brunton Hall   55.94400897108537   
5806         2268              Picady Place   55.95653524179326   

                longitude  
index                      
0      -3.192652969253117  
1      -3.196283585062929  


In [9]:
# All stations. There are 200 stations all together at least once used to start or at least once to end.
stations = pd.concat([end,start]).drop_duplicates().sort_values('station_id')
print("All stations at least once used to start or once to end:")
print('\n',stations)

All stations at least once used to start or once to end:

        station_id              station_name            latitude  \
index                                                             
27            171             George Square   55.94308406301113   
48            183           Waverley Bridge   55.95134439884462   
14            189             City Chambers   55.95010933017899   
135           225            Waverley Court   55.95173449698344   
1278          241                     Depot   55.97237325804326   
...           ...                       ...                 ...   
10024        1877         Port Edgar Marina  55.992957267668345   
855          2259          Leith Walk North   55.96791807044289   
14980        2263          Musselburgh Lidl   55.94388031687606   
21035        2265  Musselburgh Brunton Hall   55.94400897108537   
5806         2268              Picady Place   55.95653524179326   

                longitude  
index                      
27     -3.188

According to stations names some of them should be temporary: Launch Day Event, Royal Highland Show - East Gate (19th to 23rd June), Royal Highland Show - West Gate (19th to 23rd June), 
Ingliston Park and Ride (19th to 23rd June), Meadows - Edinburgh Climate Festival, Holyrood Park - Woman's Tour Of Scotland (Event 11/08/19), Pleasance - Edinburgh University Sports Fair,
HSBC UK Lets Ride - Meadows Event, Cycling Scotland Conference.



In [6]:
# Temporary stations time frame.
(edbikes_df.query("start_station_id in (267,980,981,982,991,1032,1057,1094,1740) or end_station_id in (267,980,981,982,991,1032,1057,1094,1740)")
 .sort_values('start_station_id')
)

,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
index,,,,,,,,,,,,,
42,2018-09-17 09:57:10,2018-09-17 10:11:39,869,189,City Chambers,City Chambers Quadrangle,55.95010933017899,-3.190257770093808,267,Launch Day Event,The Meadows,55.942551339390945,-3.191380756192416
29,2018-09-17 09:41:25,2018-09-17 10:05:46,1461,189,City Chambers,City Chambers Quadrangle,55.95010933017899,-3.190257770093808,267,Launch Day Event,The Meadows,55.942551339390945,-3.191380756192416
31,2018-09-17 09:46:36,2018-09-17 10:06:08,1171,189,City Chambers,City Chambers Quadrangle,55.95010933017899,-3.190257770093808,267,Launch Day Event,The Meadows,55.942551339390945,-3.191380756192416
32,2018-09-17 09:47:06,2018-09-17 09:58:24,677,189,City Chambers,City Chambers Quadrangle,55.95010933017899,-3.190257770093808,267,Launch Day Event,The Meadows,55.942551339390945,-3.191380756192416
34,2018-09-17 09:47:31,2018-09-17 10:07:39,1208,189,City Chambers,City Chambers Quadrangle,55.95010933017899,-3.190257770093808,267,Launch Day Event,The Meadows,55.942551339390945,-3.191380756192416
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10297,2019-09-22 10:40:45,2019-09-22 11:15:32,2086,1094,HSBC UK Lets Ride - Meadows Event,20 point virtual docking station for UK lets Ride Event,55.939977758149666,-3.189862157672223,1094,HSBC UK Lets Ride - Meadows Event,20 point virtual docking station for UK lets Ride Event,55.939977758149666,-3.189862157672223
10262,2019-09-22 10:11:54,2019-09-22 10:58:36,2801,1094,HSBC UK Lets Ride - Meadows Event,20 point virtual docking station for UK lets Ride Event,55.939977758149666,-3.189862157672223,1094,HSBC UK Lets Ride - Meadows Event,20 point virtual docking station for UK lets Ride Event,55.939977758149666,-3.189862157672223
873,2019-11-04 16:04:18,2019-11-04 16:19:15,897,1102,Haymarket Station,Haymarket Station,55.94556945067266,-3.21818468327956,1740,Cycling Scotland Conference,Murrayfield Stadium,55.940886347739024,-3.240777565829148


Temporary stations were used in 2018 and 2019.

In [28]:
# We need to decide how to identify Active stations.
# Active stations  will be all stations (Start stations and End stations) used last year from 30.6.2020 until the last record.


In [3]:
# Start stations used last year.
start_active = (edbikes_df.query("started_at >= '2020-06-30' or ended_at >= '2020-06-30'")[['start_station_id','start_station_name']]
 .drop_duplicates(subset='start_station_id').rename(columns={'start_station_id':'station_id','start_station_name':'station_name'})
)

In [4]:
# End stations used last year.
end_active = (edbikes_df.query("started_at >= '2020-06-30' or ended_at >= '2020-06-30'")[['end_station_id','end_station_name']]
 .drop_duplicates(subset='end_station_id').rename(columns={'end_station_id':'station_id','end_station_name':'station_name'})
)

In [5]:
# All stations used last year: Active stations.
active_stations = pd.concat([end_active,start_active]).drop_duplicates().sort_values('station_id')
active_stations.values[:,0]

array([171, 183, 189, 225, 246, 247, 248, 249, 250, 251, 252, 253, 254,
       256, 258, 259, 260, 262, 264, 265, 285, 289, 296, 299, 349, 358,
       1017, 1019, 1024, 1025, 1028, 1030, 1031, 1038, 1039, 1042, 1050,
       1051, 1052, 1055, 1090, 1091, 1092, 1093, 1096, 1097, 1098, 1102,
       1720, 1721, 1722, 1725, 1726, 1727, 1728, 1729, 1730, 1731, 1737,
       1738, 1739, 1743, 1744, 1745, 1747, 1748, 1749, 1752, 1753, 1754,
       1756, 1757, 1758, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770,
       1798, 1799, 1800, 1807, 1808, 1809, 1813, 1814, 1815, 1818, 1819,
       1820, 1821, 1822, 1823, 1824, 1859, 1860, 1864, 1865, 1866, 1868,
       1869, 1870, 1871, 1874, 1877, 2259, 2263, 2265, 2268], dtype=object)

In [10]:
# Inactive stations: all stations not used since June 30, 2020.
inactive_stations = pd.concat([stations['station_id'],active_stations['station_id'],active_stations['station_id']]).drop_duplicates(keep=False)
inactive_stations.values

array([ 241,  242,  255,  257,  261,  266,  267,  273,  275,  277,  280,
        284,  290,  297,  340,  341,  342,  343,  344,  345,  346,  347,
        350,  351,  352,  353,  354,  355,  356,  357,  359,  365,  366,
        648,  820,  860,  862,  863,  864,  865,  866,  867,  868,  869,
        870,  871,  872,  873,  874,  875,  876,  877,  878,  879,  880,
        881,  882,  883,  884,  885,  887,  888,  889,  890,  891,  901,
        964,  965,  980,  981,  982,  991, 1018, 1026, 1027, 1032, 1033,
       1040, 1041, 1056, 1057, 1094, 1095, 1723, 1724, 1740, 1746, 1857],
      dtype=int64)

In [11]:
# The most frequented stations will be stations with the highest sum of starts and ends.
# For further calculations I will use only records of active stations used last year.
edbikes_df = edbikes_df[edbikes_df['start_station_id'and'end_station_id'].isin(active_stations.values[:,0])]

In [11]:
# Dataframes with sum of starts and sum of ends.
df = edbikes_df.assign(cnt=0).groupby(['start_station_id','started_at']).count()[['cnt']]
df1 = df.groupby('start_station_id').sum().rename(columns={'cnt':'sum_borrowings'}).rename_axis('station_id')
df2 = edbikes_df.assign(cnt=0).groupby(['end_station_id','ended_at']).count()[['cnt']]
df3 = df2.groupby('end_station_id').sum().rename(columns={'cnt':'sum_returns'}).rename_axis('station_id')

In [12]:
# Stations by frequency.
activity_df = df3.join(df1)
activity_df.fillna(value=0,axis=1,inplace=True)
activity_df['frequency']=activity_df.sum_returns+activity_df.sum_borrowings
activity_df.sort_values('frequency',ascending=False)

,sum_returns,sum_borrowings,frequency
station_id,,,
265,15225,15559,30784
1728,16656,11811,28467
250,15254,9389,24643
1024,8792,11042,19834
262,11940,7752,19692
...,...,...,...
1860,224,136,360
1871,122,128,250
1030,36,45,81


In [28]:
# Number of days in records edbikes_df.
days=(pd.to_datetime(edbikes_df['started_at'],format="%Y/%m/%d").dt.date.max())-(pd.to_datetime(edbikes_df['started_at'],format="%Y/%m/%d").dt.date.min())
days

datetime.timedelta(days=1019)

In [56]:
# 10 most frequent stations.
activity_df['day_frequency']= (activity_df.frequency/1019).astype(int)
(
 activity_df.merge(stations,on='station_id',how='inner')[['station_id','station_name','frequency','day_frequency']]
 .sort_values('frequency',ascending=False).head(10)
)

,station_id,station_name,frequency,day_frequency
19,265,Meadows East,29238,28
54,1728,Portobello - Kings Road,28430,27
8,250,Victoria Quay,22860,22
28,1024,Meadow Place 2,19548,19
96,1824,Duke Street,18545,18
17,262,Canonmills,18045,17
25,358,Leith Walk,17569,17
15,259,St Andrew Square,16794,16
6,248,Bristo Square,16032,15
18,264,Pollock Halls,14979,14


In [13]:
# Added station names into activity_df
activity_df=activity_df.join(stations.set_index('station_id')).reset_index().set_index('station_name')


In [14]:
# Added percentage of returns into activity_df.
activity_df = activity_df.assign(returns_percent=(activity_df.sum_returns/activity_df.frequency*100).round(2))
activity_df

,station_id,sum_returns,sum_borrowings,frequency,returns_percent
station_name,,,,,
George Square,171,6160,4921,11081,55.59
Waverley Bridge,183,8402,7357,15759,53.32
City Chambers,189,2723,4598,7321,37.19
Waverley Court,225,2378,1849,4227,56.26
Royal Commonwealth Pool,246,4090,4092,8182,49.99
...,...,...,...,...,...
Port Edgar Marina,1877,203,195,398,51.01
Leith Walk North,2259,1537,1142,2679,57.37
Musselburgh Lidl,2263,838,709,1547,54.17


In [15]:
# Added function for usage calculation.
def usage (x):
    if x > 50:
        return "rather accumulates"
    elif x < 50:
        return "rather missing"

In [16]:
# Identification of stations where bikes rather missing or rather accumulate.
activity_df.assign(use=activity_df['returns_percent'].apply(usage)).sort_values('returns_percent',ascending=False)

,station_id,sum_returns,sum_borrowings,frequency,returns_percent,use
station_name,,,,,,
Depot Virtual,299,6,2,8,75.00,rather accumulates
Ingliston Park & Ride,1860,224,136,360,62.22,rather accumulates
Ocean Terminal,285,7381,4524,11905,62.00,rather accumulates
Victoria Quay,250,15254,9389,24643,61.90,rather accumulates
Stockbridge,258,8212,5184,13396,61.30,rather accumulates
...,...,...,...,...,...,...
Thirlestane Road,1768,1255,1832,3087,40.65,rather missing
St Andrews House,256,1957,2901,4858,40.28,rather missing
Bruntsfield Links,1767,4372,6999,11371,38.45,rather missing


In [ ]:
stations.set_index('station_name')

In [59]:
stations

,station_id,latitude,longitude
station_name,,,
George Square,171,55.94308406301113,-3.188311072722399
Waverley Bridge,183,55.95134439884462,-3.191420998989883
City Chambers,189,55.95010933017899,-3.190257770093808
Waverley Court,225,55.95173449698344,-3.18417853540609
Royal Commonwealth Pool,246,55.939000247888764,-3.173923554315593
...,...,...,...
Port Edgar Marina,1877,55.992957267668345,-3.407156205559118
Leith Walk North,2259,55.96791807044289,-3.17358629706473
Musselburgh Lidl,2263,55.94388031687606,-3.066753916217862


In [37]:
!pip install geopy
from geopy.distance import geodesic
from itertools import combinations

In [41]:
combs = list(combinations(stations.index,2))

In [42]:
coords = np.array(list(combinations(stations[['latitude', 'longitude']].values, 2)))

In [45]:
coords = coords.reshape(coords.shape[0],4)

In [46]:
coords = pd.DataFrame(coords)

In [48]:
def geodesic_vec(a, b, c, d):
    rs = geodesic( (a, b), (c, d) ).kilometers
    return rs

In [49]:
distances = list()
for data,row in coords.iterrows():
    dists = np.round(geodesic_vec(row[0],row[1],row[2],row[3]),2)
    distances.append(dists)

In [53]:
distances_df = pd.DataFrame(distances,pd.Index(combs,names=['City1','City2']),columns=['distance'])

In [64]:
distances_df.head(10)

distance
City1         City2                            
George Square Waverley Bridge              0.94
              City Chambers                0.79
              Waverley Court               1.00
              Royal Commonwealth Pool      1.01
              Charlotte Square             1.56
              Bristo Square                0.33
              Fountainbridge               1.31
              Victoria Quay                3.95
              Waverley Station             1.07
              Kings Buildings 1            2.29

In [18]:
duration_df = edbikes_df[['duration']].assign(duration_min=round(edbikes_df.duration/60))